# Generación del Dataset de Evaluación (Benchmark TFG)

Este notebook tiene como único objetivo generar el archivo `bateria_preguntas_tfg.csv`.
Este archivo contendrá las preguntas de prueba y las respuestas esperadas (Ground Truth) para evaluar las diferentes arquitecturas (SLM vs RAG).

## Estructura de los Datos
El modelo solo verá la **Pregunta**. Los demás campos son para la evaluación automática posterior:
* **Tipo:** Categoría de la pregunta (Factual, Trampa, Inferencia).
* **Pregunta:** El input para el modelo.
* **Respuesta Esperada:** La verdad contra la que compararemos.
* **Keywords:** Palabras clave obligatorias que deben aparecer en la respuesta para considerarla válida automáticamente.

In [ ]:
import pandas as pd

# Definimos la batería de preguntas manualmente
datos = [
    # --- GRUPO 1: FACTUALES (La respuesta está literal en la noticia) ---
    {
        "id": 1,
        "tipo": "Factual",
        "pregunta": "¿Qué porcentaje total de aranceles impuso Trump a China en abril de 2025?",
        "respuesta_esperada": "145% (125% nuevo más 20% existente)."
    },
    {
        "id": 2,
        "tipo": "Factual",
        "pregunta": "¿Cuánto dinero perdió Elon Musk tras el anuncio de los aranceles?",
        "respuesta_esperada": "17.800 millones de dólares en dos días."
    },
    
    # --- GRUPO 2: TRAMPA (No existe la noticia -> Debe decir 'No sé') ---
    {
        "id": 3,
        "tipo": "Trampa",
        "pregunta": "¿Qué opinó el Rey de España sobre los aranceles de Trump a China?",
        "respuesta_esperada": "No tengo información suficiente en mis archivos."
    },
     {
        "id": 4,
        "tipo": "Trampa",
        "pregunta": "¿Qué dijo Elon Musk sobre los aranceles a China?",
        "respuesta_esperada": "No tengo información suficiente (la noticia habla de Europa, no China)."
    },

    # --- GRUPO 3: INFERENCIA (Requiere unir datos o entender el contexto) ---
    {
        "id": 5,
        "tipo": "Inferencia",
        "pregunta": "¿Qué relación quiere Elon Musk entre Estados Unidos y Europa según sus declaraciones en Italia?",
        "respuesta_esperada": "Quiere una zona de libre comercio con aranceles cero."
    }
]

# Convertimos a DataFrame
df_bateria = pd.DataFrame(datos)

# Mostramos una vista previa
print(f"✅ Batería creada con {len(df_bateria)} preguntas.")
display(df_bateria)